In [ ]:
# Infrastructure and Assets Data Investigation

In [56]:
# Investigate all the themes in Rise
curl -X 'GET' 'https://data.usbr.gov/rise/api/catalog-theme-type?page=1&itemsPerPage=100' -H 'accept: application/vnd.api+json' \
| jq -r '.data[].attributes | "\(._id) - \(.theme)"'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2719    0  2719    0     0   8301      0 --:--:-- --:--:-- --:--:--  8314
1 - Water
2 - Water Quality
3 - Biological
4 - Environmental
5 - Infrastructure and Assets
7 - Hydropower
10 - Safety, Health, and Industrial Hygiene
11 - Hydraulic Modeling
12 - Water and Environment
13 - Environmental Issues for Water


In [57]:
# Get all generation IDs within the infrastructure and assets so we can link it to others
curl -X 'GET' \
  'https://data.usbr.gov/rise/api/catalog-record?page=1&itemsPerPage=25&themeId=5' \
  -H 'accept: application/vnd.api+json' | \
  jq -r '.data | .[0:15] | .[] | "\(.attributes.recordTitle) - \(.attributes.generationEffortId)"'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48374    0 48374    0     0  27665      0 --:--:--  0:00:01 --:--:-- 27658
Data and Report from S&T Project 21062: Atmospheric Plasma Coating Removal - 404
Data and Report from S&T Project 20081: Internal Erosion: Laboratory Testing to Identify End States in Internally Unstable Soils - 397
S&T Project 21079 Final Report: Implementation of a Laboratory Information Management System for Reclamation Infrastructure - 376
Report from S&T Project 21006: Development and Refinement of Rotor Turning Device for Safer and More Efficient Maintenance and Diagnostic Tasks - 390
Data and Reports from S&T Project 20024: Laboratory Evaluation of Field Repairable Materials and Techniques for Cavitation Damage: Phase II - 383
Report from S&T Project 20077: Open Data Pilot for Integrating BOR River and Reservoir Topographic and Sediment Data Into

In [67]:
# Investigate the generation effort schema for a particular item with id 397
curl -X 'GET' \
  'https://data.usbr.gov/rise/api/generation-effort/397' \
  -H 'accept: application/vnd.api+json' | jq "."

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   279    0   279    0     0    273      0 --:--:--  0:00:01 --:--:--   273
{
  "data": {
    "id": "/rise/api/generation-effort/397",
    "type": "GenerationEffort",
    "attributes": {
      "_id": 397,
      "generationEffort": "S&T Project 20081: Internal Erosion: Laboratory Testing to Identify End States in Internally Unstable Soils",
      "createDate": "2024-03-26T22:29:44+00:00"
    }
  }
}


In [75]:
# View the catalog record for themeId 5 where 5:= Infrastructure and Assets
curl -X 'GET' \
  'https://data.usbr.gov/rise/api/catalog-record?page=1&itemsPerPage=25&themeId=5' \
  -H 'accept: application/vnd.api+json' | jq -r '.data[:3]'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48374    0 48374    0     0  77367      0 --:--:-- --:--:-- --:--:-- 77274
[
  {
    "id": "/rise/api/catalog-record/8029",
    "type": "CatalogRecord",
    "attributes": {
      "_id": 8029,
      "recordTitle": "Data and Report from S&T Project 21062: Atmospheric Plasma Coating Removal",
      "recordDescription": "Two rounds of testing to evaluate and compare atmospheric plasma removal of coal tar enamel (CTE) to traditional abrasive blast processes were performed. Test samples were prepared as steel coupons measuring 3 inch x 6 inch x 1/8 inch with or without CTE coating on each, depending on the test parameter to be determined. Each round of testing was designed with separate test groups involving different surface preparation protocols, including one or a combination of atmospheric plasma, abrasive blasting, bristle blas

In [78]:
# Get the list of different catalog items associated with the theme 5. We can theme use these catalog items to link to modelRuns
curl -X 'GET' \
  'https://data.usbr.gov/rise/api/catalog-record?page=1&itemsPerPage=25&themeId=5' \
  -H 'accept: application/vnd.api+json' | jq -r '.data | .[]  | .relationships.catalogItems.data[].id'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48374    0 48374    0     0  64220      0 --:--:-- --:--:-- --:--:-- 64156
/rise/api/catalog-item/128641
/rise/api/catalog-item/128663
/rise/api/catalog-item/128684
/rise/api/catalog-item/128685
/rise/api/catalog-item/128630
/rise/api/catalog-item/128634
/rise/api/catalog-item/128635
/rise/api/catalog-item/128566
/rise/api/catalog-item/128611
/rise/api/catalog-item/128575
/rise/api/catalog-item/128576
/rise/api/catalog-item/128577
/rise/api/catalog-item/128578
/rise/api/catalog-item/72367
/rise/api/catalog-item/128526
/rise/api/catalog-item/128537
/rise/api/catalog-item/128539
/rise/api/catalog-item/11275
/rise/api/catalog-item/11416
/rise/api/catalog-item/11415
/rise/api/catalog-item/11414
/rise/api/catalog-item/11391
/rise/api/catalog-item/11379
/rise/api/catalog-item/11377
/rise/api/catalog-item/11367
/rise/api/catalog-item

In [120]:
#!/bin/bash

# Iterate through many different catalog Item IDs and check if they have modeled data associated with them
# The only way I am aware we can get result data is by getting a modelID and then querying the result
# by the model ID

# Fetch catalog item IDs associated with theme ID 5
catalog_item_ids=$(curl -s -X 'GET' \
  'https://data.usbr.gov/rise/api/catalog-record?page=1&itemsPerPage=99&themeId=5' \
  -H 'accept: application/vnd.api+json' | jq -r '.data[].relationships.catalogItems.data[].id')

# Iterate through catalog item IDs and break after the first one
for item_id in $catalog_item_ids; do
  echo "Processing Catalog Item ID: $item_id"
  echo $(basename $item_id)
  
  # Fetch catalog item details
  response=$(curl -s -X 'GET' \
    "https://data.usbr.gov/rise/api/catalog-item/$(basename $item_id)" \
    -H 'accept: application/vnd.api+json')


   echo $(echo $response | jq ".data.attributes.isModeled")
    
done


Processing Catalog Item ID: /rise/api/catalog-item/128641
128641
false
Processing Catalog Item ID: /rise/api/catalog-item/128663
128663
false
Processing Catalog Item ID: /rise/api/catalog-item/128684
128684
false
Processing Catalog Item ID: /rise/api/catalog-item/128685
128685
false
Processing Catalog Item ID: /rise/api/catalog-item/128630
128630
false
Processing Catalog Item ID: /rise/api/catalog-item/128634
128634
false
Processing Catalog Item ID: /rise/api/catalog-item/128635
128635
false
Processing Catalog Item ID: /rise/api/catalog-item/128566
128566
false
Processing Catalog Item ID: /rise/api/catalog-item/128611
128611
false
Processing Catalog Item ID: /rise/api/catalog-item/128575
128575
false
Processing Catalog Item ID: /rise/api/catalog-item/128576
128576
false
Processing Catalog Item ID: /rise/api/catalog-item/128577
128577
false
Processing Catalog Item ID: /rise/api/catalog-item/128578
128578
false
Processing Catalog Item ID: /rise/api/catalog-item/72367
72367
false
Processi